# Lab 5 - Populating and Managing Match-action Tables at Runtime

This lab describes how to populate and manage match-action tables at runtime. It then explains a tool (simple_switch_CLI) that is used with the software switch (BMv2) to manage the tables. 

<img src="./labs_files/lab1/figs/fabric_topology.png" width="550px"><br>

# Background

## Runtime 

Once a P4 program is compiled into a target-specific configuration, the output is loaded into the data plane of the device. Then, the behavior of the P4 target can be managed at runtime by the control plane via data plane Application Programming Interface (APIs). Runtime operations include inserting, updating, and deleting entries in P4 tables as well as controlling other entities of the program such as counters, meters, etc.  

Runtime APIs can be divided into program-dependent and program-independent APIs. Program-dependent APIs comprise functions whose names are derived from the P4 program itself. Thus, any changes to the P4 program would modify the names and the definitions of the APIs’ functions. Program-independent APIs comprise a set of fixed functions that are independent of the P4 program. Therefore, changes in the P4 programs do not affect those APIs. 

The control plane that manages the data plane tables and externs can be remote or local on the device. Remote control planes invoke API calls through Remote Procedure Calls (RPCs) while relying on asynchronous message frameworks such as Thrift1 and gRPC2. Such frameworks use protocol buffers (protobuf) to define service API and message, and HTTP/2.0 and TLS for transport. On the other hand, a local control plane runs on the Central Processing Unit (CPU) of the device and invokes API calls locally. It is implemented by the driver of the device. 

Figure 1 shows the runtime environment used in this lab series to control the P4 target (BMv2). The control plane uses the simple_switch_CLI tool to interact with the data plane. The simple_switch_CLI includes a program-independent CLI and a Thrift client which connects to the program-independent control server residing on the BMv2 switch. 


<img src="./labs_files/lab5/figs/runtime.png" width="350px"><br>



# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "lab5"

In [ ]:
slice = fablib.new_slice(name="lab5")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: MICH, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="550px"><br>

In [ ]:
site1='MICH'
site2='STAR'
site3='NCSA'

print (f'The selected sites are {site1}, {site2}, {site3}') 

### Step 3.3: Creating the nodes
The code below creates three nodes: server1, switch, and server2. The servers (server1 and server2) use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server3 will be created in site3

<img src="./labs_files/lab1/figs/creating_nodes.PNG" width="550px"><br>

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/lab1/figs/adding_nics.PNG" width="550px"><br>

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switch
The code below creates a node that will run the P4 switch. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

The node will be created in site2

<img src="./labs_files/lab1/figs/adding_switch.PNG" width="550px"><br>

In [ ]:
# Add a node
switch = slice.add_node(name="switch", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

<img src="./labs_files/lab1/figs/adding_switch_ports.PNG" width="550px"><br>

In [ ]:
switch_iface1 = switch.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between site1 and site2 connecting server1 and the P4 switch

<img src="./labs_files/lab1/figs/connecting_nodes_server1_switch.PNG" width="550px"><br>

In [ ]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch_iface1])

### Step 3.8: Connecting site2 and site3
Create a site-to-site network between site2 and site3 connecting the P4 switch and server2

<img src="./labs_files/lab1/figs/connecting_nodes_server2_switch.PNG" width="550px"><br>

In [ ]:
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, server2_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();

# Step 4: Installing the required packages
In this step, we will install the required packages to run the labs. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.


## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch node. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switch and execute it

In [ ]:
switch = slice.get_node(name="switch")     
switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switch, server1 and server2 nodes. This package will allow us to use the ifconfig and the arp commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switch. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/lab1/figs/interfaces.PNG" width="550px"><br>

In [ ]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net2') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

switch_iface1 = switch.get_interface(network_name='net1') 
switch_iface1_name = switch_iface1.get_device_name()
print(f'switch_iface1: {switch_iface1_name}')

switch_iface2 = switch.get_interface(network_name='net2') 
switch_iface2_name = switch_iface2.get_device_name()
print(f'switch_iface2: {switch_iface2_name}')

## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/lab1/figs/interfaces_up.PNG" width="550px"><br>

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface1_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface2_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
</ul>

<img src="./labs_files/lab1/figs/mac_addresses.PNG" width="550px"><br>

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'
switch_iface1_MAC = '00:00:00:00:00:02'
switch_iface2_MAC = '00:00:00:00:00:03'
server2_iface_MAC = '00:00:00:00:00:04'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use the network 192.168.1.0/24 between Site1 and Site2. We will assign the IP address 192.168.1.10 to server1's interface and 192.168.1.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_1.PNG" width="550px"><br>

In [ ]:
server1 = slice.get_node(name="server1")     

server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} {switch_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} hw ether {switch_iface1_MAC}')

## Step 5.5: Configuring the IP and MAC addresses on switch_iface2 and server2_iface

We will use the network 192.168.2.0/24 between Site2 and Site3. We will assign the IP address 192.168.2.10 to server2's interface and 192.168.2.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_2.PNG" width="550px"><br>

In [ ]:
server2 = slice.get_node(name="server2")     

server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} {switch_ip2}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} hw ether {switch_iface2_MAC}')

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

In [ ]:
stdout, stderr = switch.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 </li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 </li>
</ul>

<img src="./labs_files/lab1/figs/routing.PNG" width="550px"><br>

In [ ]:
gw1 = switch_ip1.split('/')[0]
gw2 = switch_ip2.split('/')[0]
stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')

## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1 and server2. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch_iface2_MAC}')

# Step 7: Navigating the switch’s CLI
   
This section demonstrates how to navigate the switch’s CLI using the simple_switch_CLI tool. This tool is used to manage P4 objects at runtime. This tool works with the BMv2 software switch. Other targets have their own tools (e.g., Intel Tofino targets use the Barefoot Runtime). 


## Step 7.1: Uploading the P4 program to the switch

In this step, we upload the P4 program to the switch, compile it, and start the switch daemon. 

The P4 program [basic.p4](labs_files/lab5/src/basic.p4) is located under lab_files/lab5/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab5/src', '/home/ubuntu/lab5')

## Step 7.2: Compiling and running the P4 program

In this step, we will use the p4c compiler to compile the program. Then, we will start the switch daemon.

Launch a new terminal by clicking on "File" -> "New" -> "Terminal".

<img src="./labs_files/lab5/figs/terminal.gif" width="600px"><br>

Copy the output of the command below and paste into the terminal to enter to the switch.

In [ ]:
switch.get_ssh_command()

Run the following commands in the terminal:
    
    sudo su
    p4c lab5/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json &
    
<img src="./labs_files/lab5/figs/daemon.png" width="600px"><br>

## Step 7.3: Navigating the CLI

In the terminal you started in the previous step, press Enter to return to the CLI, then write the following command to start the CLI.
    
    simple_switch_CLI
    
<img src="./labs_files/lab5/figs/switch_CLI.png" width="650px"><br>

<hr>

Type a question mark (?) to see the available commands in the tool

    ?


<img src="./labs_files/lab5/figs/question.png" width="650px"><br>

You can use the TAB key to autocomplete a command.

<hr>

To get help on a specific command, type help <topic>, where <topic> is the command that you would like to explore. For example, to see the syntax of table_dump, type the following:
    
    help table_dump 

<img src="./labs_files/lab5/figs/help_table_dump.png" width="480px"><br>

<hr>

## Step 7.4: Displaying ports, tables, and actions

To display the list of ports in our switch and their mapping to Linux interface names, type the following command:
    
    show_ports

<img src="./labs_files/lab5/figs/show_ports.png" width="550px"><br>

<hr>

To display the list of tables defined in the P4 program, type the following command:

    show_tables

<img src="./labs_files/lab5/figs/show_tables.png" width="600px"><br>

<hr>

List the actions defined in the P4 program by issuing the command below:

    show_actions

<img src="./labs_files/lab5/figs/show_actions.png" width="600px"><br>

Notice that the MyIngress.drop action does not have any action data whereas the action MyIngress.forward modifies the destination MAC address (i.e., dstAddr(48)) and the egress port (i.e., port(9)).

<hr>

To display basic information about the switch, type the following command:

    switch_info

<img src="./labs_files/lab5/figs/switch_info.png" width="650px"><br>

<hr>

To display the time since the switch was turned on, type the following command:

    get_time_elapsed

<img src="./labs_files/lab5/figs/get_time_elapsed.png" width="580px"><br>

The command above displays the time since the switch was turned on in microseconds.

## Step 7.5: Populating match-action tables using the switch’s CLI

This section demonstrates how to manage and populate the tables using the switch CLI tool. 

To display information about a table in the P4 program, type the following command:

    table_info MyIngress.ipv4_host
    
<img src="./labs_files/lab5/figs/table_info.png" width="580px"><br>

<hr>

Issue the following command to display the actions corresponding to a table. The output shows the actions contained in the table MyIngress.ipv4_host:

    table_show_actions MyIngress.ipv4_host
 
<img src="./labs_files/lab5/figs/table_show_action.png" width="580px"><br>

<hr>

Dump the entries of MyIngress.ipv4_host table by issuing the following command. 

    table_dump MyIngress.ipv4_host
    
<img src="./labs_files/lab5/figs/table_dump.png" width="500px"><br>

The output above shows that the table has not been populated. 

<hr>

Display the number of rules/entries in the MyIngress.ipv4_host table by issuing the following command. The output will show that there are no entries added so far.
    
    table_num_entries MyIngress.ipv4_host
    
<img src="./labs_files/lab5/figs/table_num_entries.png" width="520px"><br>

<hr>

Issue the following command to display the syntax of table_add. 

    help table_add
    
<img src="./labs_files/lab5/figs/help_table_add.png" width="800px"><br>

The parameters of the table_add can be summarized as follows:

<ul>
    <li> &lt;table_name&gt;: name of the P4 table that we would like to add rules to. The list of tables can be displayed using the show_tables command.</li>
    <li> &lt;action name&gt;: the action associated with the entry.</li>
    <li> &lt;match fields&gt;: the key used to match against the incoming packet.</li>
    <li> &lt;action parameters&gt;: the parameters associated with the entry.</li>
    <li> [priority]: the priority of the entry. </li>
</ul>
    
<hr>

Add an entry/rule to the table MyIngress.ipv4_host by issuing the following command. 

    table_add MyIngress.ipv4_host MyIngress.forward 30.0.0.1 => 00:00:00:00:00:03 2
    
<img src="./labs_files/lab5/figs/table_add.png" width="670px"><br>

The output shows the details of the new table entry. The match key is 0x1e:00:00:01 (i.e., the hexadecimal value of the IP address 30.0.0.1) and the lookup mechanism is exact. The action executed when this entry is hit will be the one defined in MyIngress.forward. The action data associated with the entry is the MAC address of the destination host (i.e., 00:00:00:00:00:03) and the egress port (i.e., 00:02).

<hr>

Issue the following command to show the entries in the table MyIngress.ipv4_host. 

    table_dump MyIngress.ipv4_host

<img src="./labs_files/lab5/figs/table_dump2.png" width="500px"><br>

<hr>

Display the number of entries in the table MyIngress.ipv4_host by typing the following command.

    table_num_entries MyIngress.ipv4_host

<img src="./labs_files/lab5/figs/table_num_entries2.png" width="500px"><br>

<hr>

We can also display the entry in a table by using its match key as follows.

    table_dump_entry_from_key MyIngress.ipv4_host 0x1e000001

<img src="./labs_files/lab5/figs/table_dump_entry.png" width="500px"><br>

Match-action tables can contain too many entries and dumping the whole table will produce a large output that is hard to read.

<hr>

Another way to display the entry in a table is by specifying the entry handle, which in this case is 0. Issue the following command to show the table entry using the handle of the entry.

    table_dump_entry MyIngress.ipv4_host 0
    
<img src="./labs_files/lab5/figs/table_dump_entry_2.png" width="500px"><br>

<hr>

Issue the following command to modify an existing entry.

    table_modify MyIngress.ipv4_host MyIngress.forward 0 00:00:00:00:00:05 5
 
<img src="./labs_files/lab5/figs/table_modify.png" width="600px"><br>

The parameters of table_modify are described as follows:

<ul>
    <li>MyIngress.ipv4_host: refers to the table that implements an exact lookup.</li>
    <li>MyIngress.forward: specifies the action.</li>
    <li>0: the table’s entry handle.</li>
    <li>00:00:00:00:00:05: the new MAC address.</li>
    <li>5: the new egress port.</li>
</ul>

<hr>

Dump the content of the table MyIngress.ipv4_host by typing the following command.
    
    table_dump_entry MyIngress.ipv4_host 0
    
<img src="./labs_files/lab5/figs/table_dump_entry_3.png" width="500px"><br>

The output shows the details of the modified entry. The match key is 0x1e:00:00:01 (i.e., the hexadecimal value of the IP address 30.0.0.1) and the lookup mechanism is exact. The action executed when this entry is hit will be the one defined in MyIngress.forward. The action data specifies 5 (i.e., the hexadecimal value 00:00:00:00:00:05) as the new destination MAC address and  port 5 as the egress.

<hr>

Add an entry/rule to the MyIngress.ipv4_lpm table by issuing the following command. 

    table_add MyIngress.ipv4_lpm MyIngress.forward 20.0.0.0/8 => 00:00:00:00:00:02 1
 
<img src="./labs_files/lab5/figs/table_add_lpm.png" width="670px"><br>

The parameters of table_add are described as follows:

<ul>
    <li>MyIngress.ipv4_lpm: refers to the table that implements LPM lookup.</li>
    <li>MyIngress.forward: specifies the action.</li>
    <li>20.0.0.0/8: is entry’s key.</li>
    <li>00:00:00:00:00:02: the destination MAC address.</li>
    <li>1: specifies the egress port.</li>
</ul>

<hr>

To delete a specific entry in a P4 table, type the following command

    table_delete MyIngress.ipv4_lpm 0

<img src="./labs_files/lab5/figs/table_delete.png" width="500px"><br>

This command deletes the entry with the handle 0 in the MyIngress.ipv4_lpm table.

<hr>

It is also possible to delete all entries from a match action table by issuing the following command.

    table_clear MyIngress.ipv4_lpm

<img src="./labs_files/lab5/figs/table_clear.png" width="500px"><br>

<hr>

Verify that the table MyIngress.ipv4_lpm is cleared by issuing the following command. The output will show that the table MyIngress.ipv4_lpm is empty.

    table_num_entries MyIngress.ipv4_lpm
 
<img src="./labs_files/lab5/figs/table_num_entries_3.png" width="430px"><br>


## Step 8: Delete the Slice

This concludes Lab 5. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="lab5")
slice.delete()